In [ ]:
from glob import glob
import xarray as xr
import logging
import os
from datetime import datetime, timezone
# We need to import every plugin type that we are going to use
from geoips.interfaces import readers, algorithms, interpolators, sectors, colormappers, output_formatters, gridline_annotators, feature_annotators

LOG = logging.getLogger(__name__)

# We use the convention: {plugin_type}.get_plugin({plugin_name}) to load the plugins at runtime

# Since we are reading abi data in a netCDF format, we will use this reader
abi_reader = readers.get_plugin("abi_netcdf")
# Since we will be reading a single channel, we want to use the single_channel algorithm
single_channel_algorithm = algorithms.get_plugin("single_channel")
# We want to use Nearest Neighbor resampling for this dataset, so we will use the interp_nearest interpolator plugin
interp_nearest = interpolators.get_plugin("interp_nearest")
# We want to restrict our output to just the CONUS sector
conus_sector_plugin = sectors.get_plugin("conus")

# We read the environment variable calling os.environ['GEOIPS_TESTDATA_DIR']
in_fpath = f"{os.environ['GEOIPS_TESTDATA_DIR']}/test_data_noaa_aws/data/goes16/20210929/0000/*.nc"
in_fpaths = glob(in_fpath)

In [ ]:
# The conus_sector_plugin contains an area_definition.
# We want to use that area_definition when calling our reader.
conus_sector_area = conus_sector_plugin.area_definition

# First we need to read the abi data for channel 14 Brightness Temperature
xdict = abi_reader(fnames=in_fpaths, area_def=conus_sector_area, chans=["B14BT"])

# Since we passed an area_def for "conus", that is the name of the Xarray Object we want to call.
conus_xobj = xdict["conus"]

# Let's look at what conus_xobj contains within it:
conus_xobj

In [ ]:
# Now let us apply the Nearest Neighbor interpolation to our data
output_dataset = interp_nearest(conus_sector_area, conus_xobj, None, ["B14BT", "longitude", "latitude"])

# Let's look at our output_dataset
output_dataset

In [ ]:
channel_14_bt = output_dataset["B14BT"].data

# Now let us apply the single channel algorithm to our channel 14 Brightness Temperature data
algorithm_output = single_channel_algorithm([channel_14_bt], output_data_range=[-90.0, 30.0], input_units="Kelvin", output_units="celsius")

output_dataset["Infrared"] = xr.DataArray(algorithm_output)

# Let's look at this dataset now that we have created it
output_dataset

In [ ]:
# We need a colormapper to tell matplotlib what colors we want to use
ir_colormapper = colormappers.get_plugin("Infrared")

ir_color_dict = ir_colormapper()

In [ ]:
# We need to tell matplotlib how we want the resulting plot to appear
# using a feature_annotator and a gridline_annotator.
# This time we will use the default versions of these plugins.
local_feature_annotator = feature_annotators.get_plugin("default")

local_gridline_annotator = gridline_annotators.get_plugin("default")

In [ ]:
#-------------------#
# imagery_annotated #
#-------------------#

# Now that we have all the setup completed, we can call our output formatter.
# Let's start by loading the imagery_annotated plugin
img_ann_output_formatter = output_formatters.get_plugin("imagery_annotated")


formatter_result_dict = {}
timestamp = datetime.strftime(datetime.now(timezone.utc), "%Y%m%d%H%M%S")
out_fpath = f"{os.environ['GEOIPS_OUTDIRS']}/abi_infrared_annotated_test_{timestamp}.png"

# And then we call the plugin
success_outputs = img_ann_output_formatter(conus_sector_area,
                                           output_dataset,
                                           "Infrared",
                                           [out_fpath],
                                           mpl_colors_info=ir_color_dict,
                                           feature_annotator=local_feature_annotator,
                                           gridline_annotator=local_gridline_annotator,
                                           output_dict=formatter_result_dict)

# Let's print the path to the image on your disk
success_outputs

In [ ]:
#---------------#
# imagery_clean #
#---------------#

# We can also create clean imagery
img_clean_output_formatter = output_formatters.get_plugin("imagery_clean")

out_fpath = f"{os.environ['GEOIPS_OUTDIRS']}/abi_infrared_clean_test_{timestamp}.png"

success_outputs = img_clean_output_formatter(conus_sector_area,
                                             output_dataset,
                                             "Infrared",
                                             [out_fpath],
                                             mpl_colors_info=ir_color_dict)

success_outputs

In [ ]:
#--------#
# NetCDF #
#--------#

# We can also put our final data into a NetCDF file
ncdf_output_formatter = output_formatters.get_plugin("netcdf_geoips")

out_fpath = f"{os.environ['GEOIPS_OUTDIRS']}/abi_infrared_xarray_test_{timestamp}.nc"

success_outputs = ncdf_output_formatter(output_dataset, ["Infrared"], [out_fpath])
success_outputs